BigQueryの使用状況を返します。

# Datasetの容量を調べる

In [18]:
import pandas as pd

DATASET = 'Acknowledgement'

sql = f"""
    SELECT
    SUM(size_bytes)  / 1000000000 AS GBs,
    SUM(row_count) AS Records
    FROM `{DATASET}.__TABLES__`
"""

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql, dialect='standard')
# df = pandas.read_gbq(sql, project_id='research-megi', dialect='standard', use_bqstorage_api=True)
df

,GBs,Records
0,0.033576,677433


# ユーザ毎のクエリ処理量の確認

In [48]:
sql = f"""
    SELECT
     user_email,
     SUM(total_bytes_processed) AS total_bytes_processed,
     -- Byte to TB
     SUM(total_bytes_processed) / 1024 / 1024 / 1024 /1024 AS total_TB_processed,
     -- TB to Dollar
     SUM(total_bytes_processed) / 1024 / 1024 / 1024 /1024 * 6.0 AS Charges_Dollar,
    FROM
    `region-asia-northeast1`.INFORMATION_SCHEMA.JOBS_BY_PROJECT
    WHERE
     DATE(creation_time) BETWEEN DATE_ADD(CURRENT_DATE('Asia/Tokyo'), INTERVAL -30 DAY ) AND CURRENT_DATE('Asia/Tokyo')
    GROUP BY 1
    ORDER BY 2 DESC
"""

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql, dialect='standard')
# df = pandas.read_gbq(sql, project_id='research-megi', dialect='standard', use_bqstorage_api=True)

df.loc['TOTAL'] = df.sum().values
df

,user_email,total_bytes_processed,total_TB_processed,Charges_Dollar
0,kusukun7@gmail.com,115352242,0.000105,0.000629
1,bq-connector@research-megi.iam.gserviceaccount...,19228026,0.000017,0.000105
TOTAL,kusukun7@gmail.combq-connector@research-megi.i...,134580268,0.000122,0.000734
